In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 7.9 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
     |████████████████████████████████| 59 kB 7.5 MB/s 
     |████████████████████████████████| 3.3 MB 56.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:
logits

tensor([[[ -35.2362,  -35.3266,  -38.9753,  ...,  -44.4645,  -43.9974,
           -36.4580],
         [-112.6171, -114.5831, -116.5724,  ..., -119.0128, -118.8059,
          -111.6917],
         [ -88.7435,  -89.8643,  -93.1977,  ...,  -92.3839,  -96.1782,
           -92.1273],
         [ -85.1646,  -88.3379,  -92.8703,  ...,  -99.8017,  -94.7657,
           -90.9330],
         [-116.7280, -119.3950, -121.7259,  ..., -129.1003, -124.6102,
          -121.6092],
         [ -77.4425,  -80.4462,  -88.0497,  ...,  -96.2564,  -93.6345,
           -84.0666]]], grad_fn=<UnsafeViewBackward0>)

In [ ]:
loss

tensor(3.9902, grad_fn=<NllLossBackward0>)

In [ ]:
outputs.values

<function CausalLMOutputWithCrossAttentions.values>

In [ ]:
inp = tokenizer('Hello are you ready for fight?')

In [ ]:
inp.input_ids

[15496, 389, 345, 3492, 329, 1907, 30]

## Title generator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import string
import urllib.request
import pickle
%matplotlib inline

In [ ]:
import csv
import itertools
import operator
import nltk
import sys
from datetime import datetime

In [ ]:
def download_data(url, save_location):
    """
    Download data to be used as corpus
    """
    print('Beginning file download...')  
    urllib.request.urlretrieve(url,save_location)
    print("Downloaded file, saving to:",save_location)

def load_data(save_location):
    """
    Load data from Textfile
    """
    file = open(save_location,"r")
    data = file.read()
    return data

def avg_char_per_title(data):
    """
    Calculate the average number of chars in a title for the sequence length
    """
    lines = data.split("\n")
    line_lengths = np.zeros(len(lines))
    for i,line in enumerate(lines):
        line_lengths[i] = len(line)
    return np.average(line_lengths)
        

def save_object(obj, filename):
    """
    Save an object - used to save models
    """
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, -1)

In [ ]:
# Change the URL to whatever text you want to train with
url = "https://gist.githubusercontent.com/AngusTheMack/defadcbc503e2d625720661e9893ff0a/raw/bb978a5ef025ff104009ab8139da4a0b7367992f/Titles.txt"

# Save Location will be used to load the data in
save_location = "Titles.txt" # either the name of the file downloaded with the URL above, or the location of your own file to load in

In [ ]:
# Downloads the data, and loads it in
download_data(url,save_location)
data = load_data(save_location)

Beginning file download...
Downloaded file, saving to: Titles.txt


In [ ]:
# Print first 100 characters of the data
print(data[:100])

Scikit-learn: Machine learning in Python
Pattern recognition and machine learning
Gaussian processes


In [ ]:
def clean_text(data):
    """
    Removes non essential characters in corpus of text
    """
    data = "".join(v for v in data if v not in string.punctuation).lower()
    data = data.encode("utf8").decode("ascii",'ignore')
    return data

In [ ]:
# You don't need to clean, but it can make things simpler
cleaned = clean_text(data)
print(cleaned[:100])

scikitlearn machine learning in python
pattern recognition and machine learning
gaussian processes i


In [ ]:
def unique_chars(data):
    """
    Get all unique Characters in the Dataset
    """
    return list(set(data))

In [ ]:
# Some info about the data
chars = unique_chars(cleaned)
data_size, input_size = len(cleaned), len(chars)
print("Data has %d characters, %d of them are unique" % (data_size, input_size))

Data has 64663 characters, 36 of them are unique


In [ ]:
def tokenize_chars(chars):
    """
    Create dictionaries to make it easy to convert from tokens to chars
    """
    char_to_idx = {ch:i for i,ch in enumerate(chars)}
    idx_to_char = {i:ch for i,ch in enumerate(chars)}
    return char_to_idx, idx_to_char

In [ ]:
# Create dictionaries, and display example using 11 chars
char_to_idx, idx_to_char = tokenize_chars(chars)
first_title = cleaned[:11]
print("{0:<2}|{1:<2}".format('Character', 'Index'))
print("________________")
for i in range(len(first_title)):
    char_index = char_to_idx[first_title[i]]
    print("{0:<9}|{a:d}".format(idx_to_char[char_index], a=char_to_idx[first_title[i]]))

Character|Index
________________
s        |25
c        |5
i        |34
k        |30
i        |34
t        |12
l        |23
e        |1
a        |17
r        |6
n        |27


In [ ]:
# Chops the stream of titles into an array of titles based on new line characters
titles = cleaned.split("\n")
titles[0]

'scikitlearn machine learning in python'

In [ ]:
unknown_token = "UNKNOWN_TOKEN"
title_start_token = "SENTENCE_START"
title_end_token = "SENTENCE_END"

In [ ]:
# Add the start and end token to the title
titles = ["%s %s %s" % (title_start_token, x, title_end_token) for x in titles]

In [ ]:
# Ensure that nltk has the punkt package
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tokenized_titles = [nltk.word_tokenize(t) for t in titles]

In [ ]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_titles))
print("Found %d unique words tokens." % len(word_freq.items()))

Found 1841 unique words tokens.


In [ ]:
vocabulary_size=2000#len(word_freq.items())
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [ ]:
print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

Using vocabulary size 2000.
The least frequent word in our vocabulary is 'ethical' and appeared 1 times.


In [ ]:
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_titles):
    tokenized_titles[i] = [w if w in word_to_index else unknown_token for w in sent]

In [ ]:
print("\nExample sentence: '%s'" % titles[0])
print("\nExample sentence after Pre-processing: '%s'" % tokenized_titles[0])


Example sentence: 'SENTENCE_START scikitlearn machine learning in python SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'scikitlearn', 'machine', 'learning', 'in', 'python', 'SENTENCE_END']'


In [ ]:
# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_titles])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_titles])

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
# Print training data example
x_example, y_example = X_train[17], y_train[17]
print("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example))

x:
SENTENCE_START supervised machine learning a review of classification techniques
[0, 66, 3, 2, 7, 49, 6, 16, 18]

y:
supervised machine learning a review of classification techniques SENTENCE_END
[66, 3, 2, 7, 49, 6, 16, 18, 1]


In [ ]:
def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

In [ ]:
class RNNNumpy:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))

In [ ]:
def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]

RNNNumpy.forward_propagation = forward_propagation

def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

RNNNumpy.predict = predict

In [ ]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print(o.shape)
print(o)

(5, 2000)
[[0.00050094 0.00049585 0.00050577 ... 0.00050363 0.00049082 0.00049915]
 [0.00050011 0.00050381 0.00050253 ... 0.00050514 0.00050839 0.0005072 ]
 [0.00050025 0.00049864 0.00049696 ... 0.00049498 0.00049688 0.00050403]
 [0.00050167 0.00050213 0.00049959 ... 0.00049484 0.00050239 0.00050337]
 [0.00050468 0.00049741 0.00050422 ... 0.00050882 0.00050223 0.00051026]]


In [ ]:
predictions = model.predict(X_train[10])
print(predictions.shape)
print(predictions)

(5,)
[1755  202    3 1314 1300]


In [ ]:
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N

RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

In [ ]:
# Limit to 1000 examples to save time
print("Expected Loss for random predictions: %f" % np.log(vocabulary_size))
print("Actual loss: %f" % model.calculate_loss(X_train[:1000], y_train[:1000]))

Expected Loss for random predictions: 7.600902


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


Actual loss: 7.601436


In [ ]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

RNNNumpy.bptt = bptt

In [ ]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print("Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape)))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = model.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = model.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print("Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix))
                print("+h Loss: %f" % gradplus)
                print("-h Loss: %f" % gradminus)
                print("Estimated_gradient: %f" % estimated_gradient)
                print("Backpropagation gradient: %f" % backprop_gradient)
                print("Relative Error: %f" % relative_error)
                return 
            it.iternext()
        print("Gradient check for parameter %s passed." % (pname))

RNNNumpy.gradient_check = gradient_check

# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
word_model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
word_model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.
Gradient Check ERROR: parameter=U ix=(0, 0)
+h Loss: 30.432536
-h Loss: 30.432536
Estimated_gradient: 0.000000
Backpropagation gradient: -0.177072
Relative Error: 1.000000


In [ ]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

RNNNumpy.sgd_step = numpy_sdg_step

In [ ]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [ ]:
np.random.seed(10)
word_model = RNNNumpy(vocabulary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

100 loops, best of 5: 7.29 ms per loop


In [ ]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
losses = train_with_sgd(model, X_train[:1000], y_train[:1000], nepoch=100, evaluate_loss_after=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


2021-11-30 09:59:51: Loss after num_examples_seen=0 epoch=0: 7.601436
2021-11-30 10:00:06: Loss after num_examples_seen=950 epoch=1: 5.101324
2021-11-30 10:00:21: Loss after num_examples_seen=1900 epoch=2: 4.884683
2021-11-30 10:00:36: Loss after num_examples_seen=2850 epoch=3: 4.782096
2021-11-30 10:00:51: Loss after num_examples_seen=3800 epoch=4: 4.696497
2021-11-30 10:01:06: Loss after num_examples_seen=4750 epoch=5: 4.617869
2021-11-30 10:01:20: Loss after num_examples_seen=5700 epoch=6: 4.550876
2021-11-30 10:01:35: Loss after num_examples_seen=6650 epoch=7: 4.489871
2021-11-30 10:01:50: Loss after num_examples_seen=7600 epoch=8: 4.432672
2021-11-30 10:02:05: Loss after num_examples_seen=8550 epoch=9: 4.378976
2021-11-30 10:02:20: Loss after num_examples_seen=9500 epoch=10: 4.327513
2021-11-30 10:02:34: Loss after num_examples_seen=10450 epoch=11: 4.277692
2021-11-30 10:02:49: Loss after num_examples_seen=11400 epoch=12: 4.230520
2021-11-30 10:03:04: Loss after num_examples_seen=

In [ ]:
sentence_start_token = 'sentence_start'
sentence_end_token = 'sentence_end'

In [ ]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[title_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[title_end_token]:
        next_word_probs = model.forward_propagation(new_sentence)
        #print(next_word_probs[0][-1])
        #print(max(next_word_probs[0][-1]))
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[0][-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

num_sentences = 15
senten_min_length = 5

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print(" ".join(sent).title())

Two Berlin A Machine Learning To Frontier Retrieval
Machine Learning Of Medical Microscopy Analytics In Functions
A Machinelearning Approach To Speech Classifying Expression On Algorithms By Identification
Towards Classification Of Data Calls In A Induction Learning The Image Output
A Framework Studies For Machinelearning Class And Machine Learning Algorithms
Automated Dependency Of Imbalanced Modeling Towards Up Using Machine Learning
Predicting Mutants Matlablike Learn Human Parallel And The Classificationa Profiling Machine
Global Extreme Learning Machine For The The
Multimodal Pharmaceutical 00 Analysis Through Routine Android For Mri Machine Machine Techniques
Statistical Zoo Methods And Machine Learning
Adaptive Size Improve Matrix Modeling Using Machine Learning Using Gene Synthesis
Machine Learning Of Applied Rules Recent In Relevance Ecosystem Optimal Control Networks Challenges
On Machine Learning To Digital Pathology Assembly In Power At Programming
Machine Learning Bias Opt

## Sentences generated by the machine

In [ ]:
# Sentence_Start A Machinelearning Approach For Automated Prediction
# Sentence_Start Image Machine Learning In Forwardlooking The Practical Output
# Sentence_Start On Multiple Comparison And Machine Learning Classifiers For Information Need State Advice Methods
# Sentence_Start A Comparison Of Supervised Machine Inhibitors In Program Emotion
# Sentence_Start Cognitive Page Qlearning Using 101 Microbial In Machine Learning
# Sentence_Start A Machine Learning Intensitybased For Inductive And Text
# Sentence_Start Meta Stealthy Construction Recognizing Information Using Using Machine Learning Approach
# Sentence_Start Psychosis Shape New Classification With Program Machine Learning
# Sentence_Start Drift Machine Learning Framework To Coreference Recognition
# Sentence_Start Using Machine Learning To Financial Efficient Prediction In Domain Iirecent Output
# Sentence_Start A Machinelearning Approach For Predicting Pixel
# Sentence_Start Webwatcher Machine Learning In Interpretable
# Sentence_Start Introduction To A Machine Learning Of Decision Supply
# Sentence_Start Health Annotator Annotation And Machine Learning Based Symbolic Authorship Datasets
# Sentence_Start Machine Learning System To Fully Prediction In Mobile Behavior Analysis

# Next execution sentence generated

# Two Berlin A Machine Learning To Frontier Retrieval
# Machine Learning Of Medical Microscopy Analytics In Functions
# A Machinelearning Approach To Speech Classifying Expression On Algorithms By Identification
# Towards Classification Of Data Calls In A Induction Learning The Image Output
# A Framework Studies For Machinelearning Class And Machine Learning Algorithms
# Automated Dependency Of Imbalanced Modeling Towards Up Using Machine Learning
# Predicting Mutants Matlablike Learn Human Parallel And The Classificationa Profiling Machine
# Global Extreme Learning Machine For The The
# Multimodal Pharmaceutical 00 Analysis Through Routine Android For Mri Machine Machine Techniques
# Statistical Zoo Methods And Machine Learning
# Adaptive Size Improve Matrix Modeling Using Machine Learning Using Gene Synthesis
# Machine Learning Of Applied Rules Recent In Relevance Ecosystem Optimal Control Networks Challenges
# On Machine Learning To Digital Pathology Assembly In Power At Programming
# Machine Learning Bias Optimize Models Heterogeneous And Prediction Research Algorithms And Compu

In [ ]:
!cp '/content/Titles.txt' -d '/content/drive/MyDrive/'

In [ ]:
from keras.applications.inception_v3 import InceptionV3

inception = InceptionV3(weights='imagenet', input_shape=(220, 220, 3), include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
for layer in inception.layers:
  layer.trainable = False

In [ ]:
inception.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 220, 220, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 109, 109, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 109, 109, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

## Now using LSTM to generate title

In [1]:
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import tensorflow.keras.utils as ku 


from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import os

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
with open('/content/drive/MyDrive/Titles.txt', encoding="UTF-8") as file:
    data = file.read()

In [3]:
import string
def clean_text(data):
    """
    Removes non essential characters in corpus of text
    """
    data = "".join(v for v in data if v not in string.punctuation).lower()
    data = data.encode("utf8").decode("ascii",'ignore')
    return data

In [4]:
cleaned = clean_text(data)
print(cleaned[:5])

sciki


In [5]:
corpus = cleaned.split("\n")
print(corpus[:10])

['scikitlearn machine learning in python', 'pattern recognition and machine learning', 'gaussian processes in machine learning', 'machine learning in automated text categorization', 'machine learning', 'thumbs up sentiment classification using machine learning techniques', 'ensemble methods in machine learning', 'c4 5 programs for machine learning', 'uci machine learning repository', 'data mining practical machine learning tools and techniques']


In [6]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
print(total_words)
inp_sequences[:10]

1840


[[161, 2],
 [161, 2, 1],
 [161, 2, 1, 7],
 [161, 2, 1, 7, 137],
 [162, 42],
 [162, 42, 4],
 [162, 42, 4, 2],
 [162, 42, 4, 2, 1],
 [138, 163],
 [138, 163, 7]]

In [7]:
import numpy as np
import tensorflow.keras.utils as ku 

def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
print(max_sequence_len)

21


In [8]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

lstm_model = create_model(max_sequence_len, total_words)
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 10)            18400     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1840)              185840    
                                                                 
Total params: 248,640
Trainable params: 248,640
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm_model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [9]:
lstm_model.save('/content/drive/MyDrive/title_model.h5')

In [10]:
from keras.models import load_model

model = load_model('/content/drive/MyDrive/title_model.h5')

In [84]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        # print(token_list)

        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        # print(token_list)
        predicted = model.predict(token_list, verbose=0) # y_predict = np.argmax(model.predict(x_test), axis=-1)
        # print(predicted)
        pre = np.argmax(predicted)
        # print(pre)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
        #   print(word, index)
          if index == pre:
              output_word = word
              break
        seed_text += " "+output_word
    return seed_text.title()

In [86]:
print(generate_text("", 5, lstm_model, max_sequence_len))
print(generate_text("euclidean", 4, lstm_model, max_sequence_len))
print(generate_text("generative", 5, model, max_sequence_len))
print(generate_text("ground breaking", 5, lstm_model, max_sequence_len))
print(generate_text("new", 4, lstm_model, max_sequence_len))
# print(generate_text("understanding", 5, lstm_model, max_sequence_len))
print(generate_text("long short term memory", 6, lstm_model, max_sequence_len))
print(generate_text("LSTM", 6, lstm_model, max_sequence_len))
print(generate_text("a", 5, lstm_model, max_sequence_len))
print(generate_text("anomaly", 5, lstm_model, max_sequence_len))
print(generate_text("data", 7, lstm_model, max_sequence_len))
print(generate_text("designing", 7, lstm_model, max_sequence_len))
print(generate_text("reinforcement", 7, lstm_model, max_sequence_len))

 101 Gcc Combining Infrared Tomography
Euclidean 101 Gcc Combining Infrared
Generative Combined Flare Flare Combining Combining
Ground Breaking 101 Gcc Combining Infrared Tomography
New Flare Combining Combining Expression
Long Short Term Memory 101 Market Combined Flare Market Lowcost
Lstm Flare Market Flare Market Deploying Combining
A 101 Market 101 Gcc Infrared
Anomaly Flare Market Market Combined Flare
Data Combined Flare Combining Combining Discrimination Discrimination Genedisease
Designing 101 Gcc Combining Infrared Tomography Deploying Deploying
Reinforcement Flare Combining Combining Expression Discrimination Genedisease Genedisease


In [101]:
def generate_text11(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        # print(token_list)

        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        # print(token_list)
        predicted = model.predict(token_list == 0.99, verbose=0).astype('int32') # y_predict = np.argmax(model.predict(x_test), axis=-1)
        # print(predicted)
        pre = np.argmax(predicted)
        # print(pre)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
        #   print(word, index)
          if index == pre:
              output_word = word
              break
        seed_text += " "+output_word
    return seed_text.title()

In [103]:
print(generate_text11("python", 5, lstm_model, max_sequence_len))
print(generate_text11("falcon", 7, lstm_model, max_sequence_len))

Python     
Falcon       


In [ ]:
from keras.callbacks import EarlyStopping, 